In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import string
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

%matplotlib inline

In [2]:
# Function to help clean up data.
# Converts strings to numeric series.
# Leaves NaN as 0

def make_num(old_series):
    translator = str.maketrans('', '', string.punctuation)
    
    new_series = [str(num).translate(translator) for num in old_series]
    new_series = pd.to_numeric(new_series, errors='coerce')
    return pd.Series(new_series)

In [3]:
# Function to import a csv file into a clean dataframe

def clean_csv(csv_filename):

    crime_df = pd.read_csv(csv_filename)
    crime_df = crime_df.drop(crime_df.index[crime_df['Population'].isnull()]).reset_index(drop=True)

    clean_df = pd.DataFrame()
    clean_df['City'] = crime_df['City']
    clean_df['State'] = csv_filename.split('_')[0].capitalize()
    clean_df['Population'] = make_num(crime_df['Population'])
    
    clean_df['ViolentCrime'] = make_num(crime_df['Violent\ncrime'])
    clean_df['Robbery'] = make_num(crime_df['Robbery'])
    clean_df['Murder'] = make_num(crime_df['Murder and\nnonnegligent\nmanslaughter'])

    clean_df['Rape'] = pd.concat([make_num(crime_df['Rape\n(legacy\ndefinition)2']), 
                            make_num(crime_df['Rape\n(revised\ndefinition)1'])], axis=1).max(axis=1)

    clean_df['Assault'] = make_num(crime_df['Aggravated\nassault'])
    
    clean_df['PropertyCrime'] = make_num(crime_df['Property\ncrime'])
    clean_df['Burglary'] = make_num(crime_df['Burglary'])
    clean_df['Larceny'] = make_num(crime_df['Larceny-\ntheft'])
    clean_df['MVTheft'] = make_num(crime_df['Motor\nvehicle\ntheft'])

    clean_df.fillna(0)
    
    return clean_df

In [4]:
al_df = clean_csv('alabama_by_city_2013.csv')
ak_df = clean_csv('alaska_by_city_2013.csv')
az_df = clean_csv('arizona_by_city_2013.csv')
ar_df = clean_csv('arkansas_by_city_2013.csv')
ca_df = clean_csv('california_by_city_2013.csv')
co_df = clean_csv('colorado_by_city_2013.csv')
ct_df = clean_csv('connecticut_by_city_2013.csv')
de_df = clean_csv('delaware_by_city_2013.csv')
dc_df = clean_csv('districtofcolumbia_by_city_2013.csv')
fl_df = clean_csv('florida_by_city_2013.csv')

ga_df = clean_csv('georgia_by_city_2013.csv')
id_df = clean_csv('idaho_by_city_2013.csv')
il_df = clean_csv('illinois_by_city_2013.csv')
in_df = clean_csv('indiana_by_city_2013.csv')
ia_df = clean_csv('iowa_by_city_2013.csv')
ks_df = clean_csv('kansas_by_city_2013.csv')
kt_df = clean_csv('kentucky_by_city_2013.csv')
la_df = clean_csv('louisiana_by_city_2013.csv')
me_df = clean_csv('maine_by_city_2013.csv')
md_df = clean_csv('maryland_by_city_2013.csv')

ma_df = clean_csv('massachusetts_by_city_2013.csv')
mi_df = clean_csv('michigan_by_city_2013.csv')
mn_df = clean_csv('minnesota_by_city_2013.csv')
ms_df = clean_csv('mississippi_by_city_2013.csv')
mo_df = clean_csv('missouri_by_city_2013.csv')
mt_df = clean_csv('montana_by_city_2013.csv')
ne_df = clean_csv('nebraska_by_city_2013.csv')
nv_df = clean_csv('nevada_by_city_2013.csv')
nh_df = clean_csv('newhampshire_by_city_2013.csv')
nj_df = clean_csv('newjersey_by_city_2013.csv')

nm_df = clean_csv('newmexico_by_city_2013.csv')
ny_df = clean_csv('newyork_by_city_2013.csv')
nc_df = clean_csv('northcarolina_by_city_2013.csv')
nd_df = clean_csv('northdakota_by_city_2013.csv')
oh_df = clean_csv('ohio_by_city_2013.csv')
ok_df = clean_csv('oklahoma_by_city_2013.csv')
or_df = clean_csv('oregon_by_city_2013.csv')
pa_df = clean_csv('pennsylvania_by_city_2013.csv')
ri_df = clean_csv('rhodeisland_by_city_2013.csv')
sc_df = clean_csv('southcarolina_by_city_2013.csv')

sd_df = clean_csv('southdakota_by_city_2013.csv')
tn_df = clean_csv('tennessee_by_city_2013.csv')
tx_df = clean_csv('texas_by_city_2013.csv')
ut_df = clean_csv('utah_by_city_2013.csv')
vt_df = clean_csv('vermont_by_city_2013.csv')
va_df = clean_csv('virginia_by_city_2013.csv')
wa_df = clean_csv('washington_by_city_2013.csv')
wv_df = clean_csv('westvirginia_by_city_2013.csv')
wi_df = clean_csv('wisconsin_by_city_2013.csv')
wy_df = clean_csv('wyoming_by_city_2013.csv')

In [5]:
df_all = pd.concat([al_df, ak_df, az_df, ar_df, ca_df, co_df, ct_df, de_df, dc_df, fl_df,
                   ga_df, id_df, il_df, in_df, ia_df, ks_df, kt_df, la_df,
                   me_df, md_df, ma_df, mi_df, mn_df, ms_df, mo_df, mt_df,
                   ne_df, nv_df, nh_df, nj_df, nm_df, ny_df, nc_df, nd_df,
                   oh_df, ok_df, or_df, pa_df, ri_df, sc_df, sd_df, tn_df,
                   tx_df, ut_df, vt_df, va_df, wa_df, wv_df, wi_df, wy_df]).reset_index(drop=True)

df_all.shape

(9288, 12)

In [6]:
df_all.groupby('State').sum()

,Population,ViolentCrime,Robbery,Murder,Rape,Assault,PropertyCrime,Burglary,Larceny,MVTheft
State,,,,,,,,,,
Alabama,2392789,14324.0,3576,228,11540.0,9433.0,110445.0,26399.0,77067.0,6979
Alaska,473660,3498.0,599,16,553.0,2330.0,17776.0,1924.0,14577.0,1275
Arizona,4867945,21161.0,6132,248,15340.0,13247.0,181403.0,37936.0,129979.0,13488
Arkansas,1617882,9181.0,1892,103,874.0,6312.0,74226.0,19363.0,51346.0,3517
California,31696400,124598.0,48035,1400,6064.0,69099.0,869985.0,190417.0,539803.0,139765
Colorado,3658311,13438.0,2923,134,2291.0,8090.0,116633.0,20321.0,85753.0,10559
Connecticut,3058728,8795.0,3466,80,6730.0,4576.0,65542.0,11520.0,48240.0,5782
Delaware,252549,2148.0,687,21,113.0,1327.0,11154.0,1995.0,8576.0,583
Districtofcolumbia,646449,7880.0,3660,103,393.0,3724.0,29569.0,3314.0,23108.0,3147


In [30]:
per_capita_df = pd.DataFrame(df_all.groupby('State').apply(lambda x: x['ViolentCrime'].sum() / x['Population'].sum()))
per_capita_df.rename(columns={0:'ViolentCrimePerCap'})

,ViolentCrimePerCap
State,
Alabama,0.005986
Alaska,0.007385
Arizona,0.004347
Arkansas,0.005675
California,0.003931
Colorado,0.003673
Connecticut,0.002875
Delaware,0.008505
Districtofcolumbia,0.012190


In [31]:
per_capita_df.sort_values(by=0)

,0
State,
Vermont,0.001382
Maine,0.001526
Illinois,0.001856
Newhampshire,0.002197
Wyoming,0.002204
Utah,0.002368
Idaho,0.002479
Rhodeisland,0.002528
Newjersey,0.002700
